In [1]:
pip install pymupdf

In [2]:
pip install langchain==0.1.17 langchain-community==0.0.36 langchain-groq

In [3]:
pip install faiss-cpu pypdf sentence-transformers

In [4]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("/content/04-Policy-Manual-Apr-2016-excl-SAICA.pdf")
documents = loader.load()

for i, doc in enumerate(documents[:5]):
    print(f"Doc {i} length:", len(doc.page_content))


Doc 0 length: 1337
Doc 1 length: 1645
Doc 2 length: 1320
Doc 3 length: 2584
Doc 4 length: 1453


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=100
)
docs = text_splitter.split_documents(documents)


In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [8]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)


In [14]:
from langchain.chains.retrieval_qa.base import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)


In [17]:
response = qa_chain.invoke({
    "query": "What procedures are followed for policy violations?"
})

print(response["result"])

The procedures for policy violations are outlined in the disciplinary code, which includes the following stages:

1. An inquiry is conducted to assess the guilt of the employee.
2. The penalty is assessed separately from the question of guilt.
3. The employee is entitled to be advised of the penalty imposed.
4. If the inquiry decides that the guilt is proven and imposes the decision of dismissal, the employee will be informed both verbally and in writing.
5. The employee will also be informed of their right of appeal.

Additionally, it is mentioned that the employer has the right to discipline an employee for a transgression not listed in the disciplinary code, and that each case will be considered on its own facts and merits, taking into account mitigating and aggravating circumstances.
